# CH 14

## TOC<a id='toc'></a>
* [Ch14 Notes](#ch14_notes)
* [Ch13 Notes](#ch13_notes)

### CH14 Notes <a id='ch14_notes'></a>
[toc](#toc)
### Iterables, Iterators and Generators

* The **yield** keyword was added in python 2.2, which allows the construction of generators, which work as iterators.
* *Every generator is an iterator*: generators fully implement the iterator interface. But an iterator (as defined in the GoF book) retrieves iterms from a collection, while a generator can produce iterms "out of thin air".
    - however, python community treats iterator and generator as synomyms most of the time.
* Every collection in python is **iterable**, and iterators are used internally to support:
    - for loops
    - collection types constructions and extensions
    - looping over text files line by line
    - list, dict, and set comprehensions
    - typle unpacking
    - unpacking actual parameters with * in function call

## Sentence Take #1: A Sequence of Words
* when an interpreter needs to iterate over and object x, it automatically calls `iter(x)`. It
    1. checks whether object implements `__tier__`, and calls that to obtain an iterator
    2. if `__iter__` not implemented, but `__getitem__` is, python creates and iterator that attemps to fetch items in order starting from index 0 (why all sequences are iterable)
    3. If that fails, python raises TypeError
* from the sense of goose typing, and object is consideret iterable if it implements `__iter__` method. 
    - no subclassing or registration required because abc.Iterable implements the sibclasshook.
* but because of the getitem fallback, as of python 3.4, most accurate way to check if object is iterable is to call iter on it and handle TyperError exception.

* **iterable**: an object from which the iter function can obtain an iteratir. 
    - objects implementing an `__iter__` method returning an *iterator* are itertable
* **iterator**: ...?
        - from python docs - "An object representing a stream of data. can call next on it. Will raise StopIteration when exhausted. (and every subsequent call to next)
        - it is required to implement an `__iter__` function and return self, so that it can also be used as an iterable.

* The standard interface for an interator has two methods:
   * `__next__`: returns next available item, raising `StopIteration` when there are no more items
   * `__iter__`: Returns self; this allows the iterators to be used where an iterable is expected - for example in a for loop

```
class Iterator(Iterable):
    __slots__ = ()
    
    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted raise StopIteration'
        raise StopIteration
    
    def __iter__(self):
        return self
        
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            if (any("__next__" in B.__dict__ for B in C.__mro__) and any("__iter__" in B.__dict__ for B in C.__mro__)):
                return True
        return NotImplemented
```

From python source code: "Iterators in python aren't a matter of type but of protocol. ... Don't check the type! Use hasattr to check for both "`__iter__`" and "`__next__`" attributes instead.

There is no way to "reset" and iterator. If you need to start over, you need to call iter(...) on the iterable that built it.
    - this wont work f you built it from an iterator because it just returns self

## Sentence Take # 2: A Classic Interior
* built according to classic iterator design pattern following GoF (not idiomatic python)
* This sentence is *iterable* because it implements the `__iter__` special method, which returns a `SentenceIterator`
* writes two classes: `Sentence`  which is built with some text, and `SentenceIterator` built with a list of words (returned by sentence `__iter__`), and contains an index which gets advanced at every call of next(..)


* all iterators are iterables, but not all iterables are iterators
* it is tempting to implement `__next__` in addition to `__iter__` in Sentence, but that is a terrible idea. It is a common *anti-pattern* accordin to Alex Martelli
    - you need to be able to support multiple traversals
    - that is, you must be able to support multiple independent iterators, from same iterable instance; each with its own internal state.

## Sentence Take #3: A generator Function

```
class Sentence:

    def __init__(self, text)::
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __iter___(self):
        for word in self.words:
            yield word
        return
```

* The last return is not needed, function can just fall through.
* generator function does not need to raise StopIteration, it simply exits.

### How a Generator Function Works